<a href="https://colab.research.google.com/github/atacankavdir/academic/blob/main/volatility_baseline_modelrun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
pip install numpy pandas tensorflow arch keras sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 4.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Example usage:
# Load your data into a pandas DataFrame
data = pd.read_csv('/content/drive/My Drive/DS Masters/Data/ETH-USD.csv', index_col='Date', parse_dates=True)

In [5]:
import numpy as np
import pandas as pd
from arch import arch_model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

# Calculate returns from the price data
eth_data['returns'] = eth_data['price'].pct_change().dropna()

# Data Preparation for AP-GARCH and DFFNN
returns = eth_data['returns'].values[1:]  # remove the first NA value

# Step 1: Fit AP-GARCH Model
def fit_apgarch(returns):
    apgarch_model = arch_model(returns, vol='Garch', p=1, q=1, o=1, power=1.0)
    res = apgarch_model.fit(update_freq=5, disp='off')
    forecast = res.conditional_volatility
    return forecast

# Step 2: Prepare data for DFFNN
def prepare_data_for_dffnn(returns, forecasted_volatility):
    scaler = MinMaxScaler()
    scaled_returns = scaler.fit_transform(returns.reshape(-1, 1))
    scaled_volatility = scaler.transform(forecasted_volatility.reshape(-1, 1))

    # Create a feature set with returns and forecasted volatility
    features = np.hstack((scaled_returns[:-1], scaled_volatility[:-1]))  # shift by one to predict next step
    labels = scaled_returns[1:]

    return features, labels

# Step 3: Define DFFNN
def build_dffnn(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Main pipeline
forecasted_volatility = fit_apgarch(returns)
features, labels = prepare_data_for_dffnn(returns, forecasted_volatility)
model = build_dffnn(features.shape[1])

# Split data into training and testing sets (80-20 split for example)
split = int(len(features) * 0.8)
X_train, X_test = features[:split], features[split:]
y_train, y_test = labels[:split], labels[split:]

# Step 4: Train DFFNN
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

# Evaluation
mse = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {mse}")

# For a complete AP-DFFN model, you might iterate through the process, tweaking model parameters based on performance


NameError: name 'eth_data' is not defined